### Model Inferencing

In [ ]:
import time
import pickle
import numpy as np
import keras
import tensorflow as tf

print(f'using keras version... {keras.__version__} & tf version... {tf.__version__}')

In [ ]:
%%time
# 1: loading tokenizer
from keras.preprocessing.text import Tokenizer
TOKENIZER_MODEL = "tokenizer.pkl"
tokenizer = pickle.load(open(TOKENIZER_MODEL, "rb"))  # todo: might want to fix protocol

vocab_size = len(tokenizer.word_index) + 1
print("Total words", vocab_size)

In [ ]:
# 2: loading w2v model
from gensim.models import KeyedVectors
from keras.layers import Embedding
w2v_model = KeyedVectors.load("model.w2v", mmap='r')
W2V_SIZE = 300

embedding_matrix = np.zeros((vocab_size, W2V_SIZE))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

embedding_layer = Embedding(vocab_size, W2V_SIZE, weights=[embedding_matrix], trainable=False)

In [ ]:
# 3: loading keras model
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras import utils

model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.5))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.load_weights('model.h5')
model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
import re
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)


def process_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [ ]:
# SENTIMENT
from keras.preprocessing.sequence import pad_sequences

POSITIVE = "POSITIVE"
NEGATIVE = "NEGATIVE"
NEUTRAL = "NEUTRAL"
SENTIMENT_THRESHOLDS = (0.4, 0.7)
SEQUENCE_LENGTH = 300

def decode_sentiment(score, include_neutral=True):
    if include_neutral:        
        label = NEUTRAL
        if score <= SENTIMENT_THRESHOLDS[0]:
            label = NEGATIVE
        elif score >= SENTIMENT_THRESHOLDS[1]:
            label = POSITIVE
        return label
    else:
        return NEGATIVE if score < 0.5 else POSITIVE

def predict(text, include_neutral=True):
    start_at = time.time()
    text = process_text(text)
    # Tokenize text
    x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=SEQUENCE_LENGTH)
    # Predict
    score = model.predict([x_test])[0]
    # Decode sentiment
    label = decode_sentiment(score, include_neutral=include_neutral)

    return {"label": label, "score": float(score),
       "elapsed_time": time.time()-start_at}

In [ ]:
predict(text="I love the music", include_neutral=True)

In [ ]:
predict(text="I hate the rain", include_neutral=True)

In [ ]:
predict(text="Asco 2020 sees multiple myeloma responses deepen", include_neutral=True)

In [ ]:
inp_text = "We are heartened by progress against cancer. But we cannot overlook current events. \
            Racism cannot be ignored. It undermines public health. We commit to health equity & doing \
            our part to address inequities in research & care for every patient, everywhere, every day."

In [ ]:
predict(text=inp_text, include_neutral=True)